# A simple LLM Application with LCEL

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

## Why LangSmith
Many of the applications built with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside the chain or agent. The best way for this is LangSmith.

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [3]:
!pip install -qU langchain-groq


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-8b-8192")

`ChatModels` are instances of LangChain **Runnables**
To simply call the model, we can pass in a list of messages to the **.invoke** method.

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

In [6]:
messages  = [
    SystemMessage(content="Translate the following from English into Nepali language."),
    HumanMessage(content="Hello, how are you doing?")
]

In [7]:
model.invoke(messages)

AIMessage(content='नमस्ते, तिमी कसरी छ? (Namaste, timi kassari cha?)\n\n(Note: "नमस्ते" (Namaste) is a formal way of saying "hello" in Nepali, and "तिमी कसरी छ" (timi kassari cha?) is a way of asking "how are you doing?")', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 32, 'total_tokens': 113, 'completion_time': 0.064319251, 'prompt_time': 0.005561105, 'queue_time': None, 'total_time': 0.06988035599999999}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-7e5468e4-cc8e-4768-87e1-3d3019a7012a-0')

See the **LangSmith trace** to see the logs

## Output Parsers
To parse just the response

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [10]:
result = model.invoke(messages)
result = parser.invoke(result)

print(result)

नमस्ते, तिमी कसरी हुनुहुन्छ? (Namaste, timi kasari hunuhunchha?)

Translation:

* Namaste: Hello (a traditional Nepali greeting)
* timi: you
* kasari: how
* hunuhunchha: are you doing (literally "are you fine")

So, the entire phrase "नमस्ते, तिमी कसरी हुनुहुन्छ?" literally means "Hello, how are you?"


We can also "chain" the model with this output parser. This means this output parser will get called everytime in this chain.

In [11]:
chain = model | parser

In [12]:
messages2 = [
    SystemMessage(content="Translate the following from English into Nepali language. Just response with the translation and nothing else."),
    HumanMessage(content="Nepal is a beautiful country located in Asia continent.")
]

chain.invoke(messages2)

'नेपाल एशिया महादेशमा सुन्दर देश हो।'

## Prompt Templates
What I did above was abstraction over prompt template. Usually **messages** is constructed from a combination of user input and application logic.This application logic takes the raw user  input and transforms it into a list of messages ready to pass to the llm.

In [13]:
from langchain_core.prompts import ChatPromptTemplate

In [20]:
system_template = "Translate the following into {language} language and respond with the translation just respond with the translation and nothing else."

Now a **Prompt Template** will be a combination of the **System Template** as well as a simpler template for text

In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [16]:
result = prompt_template.invoke(
    {
        "language": "Nepali",
        "text": "Discipline is the bridge between goals and accomplishment."
    }
)
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Nepali:'), HumanMessage(content='Discipline is the bridge between goals and accomplishment.')])

In [17]:
result.to_messages()

[SystemMessage(content='Translate the following into Nepali:'),
 HumanMessage(content='Discipline is the bridge between goals and accomplishment.')]

## Chaining together components with LCEL
we can now combine this with the model and output parser from above using the `pipe (|)` operator.

In [22]:
chain = prompt_template | model | parser

In [23]:
custom_messages = {
    "language": "Nepali",
    "text": "Discipline is the bridge between goals and accomplishment."
}

response = chain.invoke(custom_messages)
print(response)

संस्कार हुन्छ काम आउने र पूरा हुने बीच को पुल हो ।


Now this can be easilt be served using `**LangServe**`